# Colab commands

In [ ]:
!cp /content/drive/MyDrive/Colab\ Notebooks/build_model/TrainValidate.py /content
!cp /content/drive/MyDrive/Colab\ Notebooks/datasets/final_features_removed.csv /content
!cp /content/drive/MyDrive/Colab\ Notebooks/datasets/final_labels_removed.csv /content
!cp /content/drive/MyDrive/Colab\ Notebooks/datasets/final_sources_removed.csv /content
!mkdir /content/models

In [ ]:
!pip install --quiet torchinfo
!pip install --quiet torch_snippets

# Import packages

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler

import torchinfo
from torch_snippets import Report
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from TrainValidate import TrainValidate, create_weighted_sampler

device = "cpu"
if torch.cuda.is_available():
    device = "cuda"
elif torch.has_mps:
    device = "mps"

/home/wl21/anaconda3/envs/deep_learning/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device

'cuda'

# Load datasets

In [3]:
features = pd.read_csv("../datasets/final_features_removed.csv")
labels = pd.read_csv("../datasets/final_labels_removed.csv")
sources = pd.read_csv("../datasets/final_sources_removed.csv")

# Create class for collecting data

In [4]:
class StatuePriceDataset(Dataset):
    def __init__(self, features, labels):
        assert len(features) == len(labels)
        self.features = features
        self.labels = labels

    def __getitem__(self, item):
        features = self.features.to_numpy()[item]
        features = torch.tensor(features).float().to(device)

        labels = self.labels.to_numpy()[item]
        price = torch.tensor([labels[1]]).float().to(device)
        status = torch.tensor([labels[0]]).float().to(device)

        features = torch.cat([features, price])

        return features, status

    def __len__(self):
        return len(self.features)

In [5]:
temp = StatuePriceDataset(features, labels)
in_features = len(temp[0][0])
in_features

52

# Build model

In [6]:
class PredictPrice(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.in_features = in_features

        self.hidden = nn.Sequential(nn.Linear(in_features, 128),
                                    nn.ReLU(),
                                    nn.Linear(128, 128),
                                    nn.ReLU(),
                                    nn.Linear(128, 256),
                                    nn.ReLU(),
                                    nn.Linear(256, 128),
                                    nn.ReLU())
        self.price = nn.Sequential(nn.Linear(128, 1),
                                   nn.ReLU())

    def forward(self, x):
        x = self.hidden(x)
        price = self.price(x)
        return price.squeeze()

class PredictStatus(nn.Module):
    def __init__(self, in_features):
        super(PredictStatus, self).__init__()

        # Load model for predicting prices and freeze its parameters
        filename = "models/all_removed_weighted_sampler_full_epoch_1000_fold_2.pth"
        self.price = torch.load(filename, map_location=torch.device("cpu"))
        for param in self.price.parameters():
            param.requires_grad = False

        self.hidden = nn.Sequential(nn.Linear(in_features+ 1, 128),
                                    nn.ReLU(),
                                    nn.Linear(128, 128),
                                    nn.ReLU(),
                                    nn.Linear(128, 256),
                                    nn.ReLU(),
                                    nn.Linear(256, 128),
                                    nn.ReLU())
        self.status = nn.Sequential(nn.Linear(128, 1),
                                    nn.Sigmoid())

    def forward(self, x):
        pred_price = self.price(x[:, :-1])[:, None]

        new_features = torch.cat([x, pred_price], 1)
        out = self.hidden(new_features)
        out = self.status(out)
        return out

In [7]:
model = PredictStatus(in_features)
torchinfo.summary(model, input_size=(16, in_features))

Layer (type:depth-idx)                   Output Shape              Param #
PredictStatus                            [16, 1]                   --
├─PredictPrice: 1-1                      [16]                      --
│    └─Sequential: 2-1                   [16, 128]                 --
│    │    └─Linear: 3-1                  [16, 128]                 (6,656)
│    │    └─ReLU: 3-2                    [16, 128]                 --
│    │    └─Linear: 3-3                  [16, 128]                 (16,512)
│    │    └─ReLU: 3-4                    [16, 128]                 --
│    │    └─Linear: 3-5                  [16, 256]                 (33,024)
│    │    └─ReLU: 3-6                    [16, 256]                 --
│    │    └─Linear: 3-7                  [16, 128]                 (32,896)
│    │    └─ReLU: 3-8                    [16, 128]                 --
│    └─Sequential: 2-2                   [16, 1]                   --
│    │    └─Linear: 3-9                  [16, 1]              

# Data standardization

In [9]:
scaler = StandardScaler()
scaler.fit(features)
features[:] = scaler.transform(features)

# Train the model
## Prepare data

In [10]:
x_train, x_test, y_train, y_test = train_test_split(features, labels, random_state=1, test_size=0.1)

# Cross validation

In [14]:
kfold = KFold(n_splits=5, shuffle=True, random_state=1)
torch.manual_seed(13)
epochs = 1000

for fold, (train_id, val_id) in enumerate(kfold.split(x_train.index)):
    train_feature, train_label = x_train.iloc[train_id], y_train.iloc[train_id]
    val_feature, val_label = x_train.iloc[val_id], y_train.iloc[val_id]
    print("\n\n-------------This is fold {}----------------".format(fold))

    train_data = StatuePriceDataset(train_feature, train_label)
    val_data = StatuePriceDataset(val_feature, val_label)
    in_features = len(train_data[0][0])

    train_sampler = create_weighted_sampler(train_feature["Sale or Let"].values)
    val_sampler = create_weighted_sampler(val_feature["Sale or Let"].values)

    train_loader = DataLoader(train_data, batch_size=16, shuffle=False, drop_last=True, sampler=train_sampler)
    val_loader = DataLoader(val_data, batch_size=16, shuffle=False, drop_last=True, sampler=val_sampler)

    model = PredictStatus(in_features).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    train_validate = TrainValidate(model, nn.BCELoss(), optimizer)
    train_validate.set_loader(train_loader, val_loader)
    train_validate.train(epochs)

    train_validate.save_model("models/status_removed_full_epoch_1000_fold_{}.pth".format(fold))



-------------This is fold 0----------------
EPOCH: 86.626	train_loss: 31.250	(311.04s - 3279.59s remaining)

KeyboardInterrupt: 